# Data Analysis (Using the PySpark Library)

- **The analysis of this dataset using the pandas library can be found here: https://github.com/AVC-prog/Data_Science_and_Analysis_with_Python_and_SQL/blob/main/Project%204%3A%20Car%20Sales%20Analysis/Project%204%20(Car%20Sales%20Analysis).ipynb**

# Prefactory Remarks

- **Create a virtual environment to download the packages**

In [ ]:
# You don't have to do this, it's just safer.

# Install virtualenv (virtual environment):

# !pip install virtualenv

# Create a virtual environment named "myenv":

# !python -m venv myenv

# Activate the virtual environment:

# myenv\Scripts\activate (Windows)
# source myenv/bin/activate (macOS/Linux)

# Upgrade pip and install essential data science libraries inside the virtual environment:

# !myenv/bin/python -m pip install --upgrade pip  
# !myenv/bin/python -m pip install numpy pandas matplotlib seaborn scikit-learn scipy statsmodels jupyterlab plotly openpyxl xlrd tensorflow keras torch torchvision pyspark ipykernel

# Add the virtual environment as a Jupyter kernel:

# !myenv/bin/python -m ipykernel install --user --name=myenv --display-name "Python (myenv)"

# Deactivate the virtual environment (Run this in the terminal):

# deactivate

- **Libraries we might need to install or upgrade**

In [ ]:
# If you don't care to create a virtual environment, here is what you need to do to download the libraries

# Run these directly in a cell to download the libraries:

#!pip install tensorflow
#!pip install pyspark
#!pip install scikit-optimize (for skopt)
#!pip install missingno
#!pip install seaborn
#!pip install numpy
#!pip install pandas
#!pip install matplotlib
#!pip install scikit-learn

# To update them, run this (with your desired library):

#!pip install --upgrade scikit-learn

- **Tips for rearranging your Notebook**

- Hold ctrl+shift and click on the various cells you want to move, then press the arrow keys to move them up or down.

## 1. Visualize the Data

- [x] **Start the PySpark session and view the data.**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *  
from pyspark.sql.types import *  

spark = SparkSession.builder.appName("Test_CS").getOrCreate()


df = spark.read.csv("/kaggle/input/car-sales/car_sales.csv", header=True, inferSchema=True)

df.show(5)

## 2. Clean the Data

- [x] **Check if there are any NaN values. Use imputation to fill them in as necessary.**

In [ ]:
# Let's put them in a dictionary, just like we've done in the pandas notebook

not_numbers = {}

for column in df.columns:
    not_numbers[column] = df.filter(df[column].isNull()).count()

print(not_numbers)

- [x] **Check if there are any missing values. (empty slots)**

In [ ]:
# Again, let's print the values out in a dictionary, just like in the pandas notebook

empty_values = {}

for column in df.columns:
    empty_values[column] = df.filter(df[column] == "").count()

print(empty_values)

- [x] **Disassociate the date column into day,month, and year.**

In [ ]:
from pyspark.sql import functions as F

df = df.withColumn("Date", F.to_date(df["Date"], "MM/dd/yyyy"))

df = df.withColumn("week_day", F.date_format(df["Date"], "EEEE"))  
df = df.withColumn("day", F.dayofmonth(df["Date"]))  
df = df.withColumn("month", F.date_format(df["Date"], "MMMM"))  
df = df.withColumn("year", F.year(df["Date"]))  

df.show(5)

- [x] **Rename the columns to more SQL-friendly names.**

In [ ]:
df = df.withColumnRenamed('Car_id', 'car_id') \
       .withColumnRenamed('Customer Name', 'customer_name') \
       .withColumnRenamed('Annual Income', 'annual_income') \
       .withColumnRenamed('Dealer_Name', 'dealer_name') \
       .withColumnRenamed('Company', 'company') \
       .withColumnRenamed('Model', 'model') \
       .withColumnRenamed('Engine', 'engine') \
       .withColumnRenamed('Transmission', 'transmission') \
       .withColumnRenamed('Color', 'color') \
       .withColumnRenamed('`Price ($)`', 'price') \
       .withColumnRenamed('Dealer_No', 'dealer_no') \
       .withColumnRenamed('Body Style', 'body_style') \
       .withColumnRenamed('Phone', 'phone') \
       .withColumnRenamed('Dealer_Region', 'dealer_region') \
       .withColumnRenamed('Date', 'date') \
       .withColumnRenamed('Gender', 'gender')

df.show(10)

- [x] **Save the clean csv file.**

In [ ]:
# output_path = "/kaggle/working/cleaned_car_sales.csv"
# df.write.option("header", "true").csv(output_path)

## 3. Exploratory Data Analysis (EDA) and Visualizations

- **The visualizations have been done in the pandas notebook, and given that the code would be the exact same here, I will exempt myself from repeating them.**

- [x] **Create a dictionary that shows the value counts for each element in each categorical column.**

In [ ]:
categorical_columns = ["Gender", "Company", "Model", "Engine", "Transmission", "Color", "Body Style", "Dealer_Region"]

category_counts = {}

for col in categorical_columns:
    counts_df = df.groupBy(col).count()
    category_counts[col] = {row[col]: row["count"] for row in counts_df.collect()}

print(category_counts)

- [x] **Most sales on weekends?**

In [ ]:
from pyspark.sql.functions import to_date, date_format, col, count

df = df.withColumn("date", to_date(col("date"), "M/d/yyyy"))
df = df.withColumn("week_day", date_format(col("date"), "EEEE"))
sales_by_weekday = df.groupBy("week_day").agg(count("*").alias("total_sales"))

sales_by_weekday.show()

- [x] **Create a new column called 'buy_prob' that is the probability of buying. It should follow the formula: $P = (\textrm{income} - 3 * \textrm{price}) / \textrm{income}$**

In [ ]:
# Income must me 3*car price for it to be considered 
# and whatever is beyond that reflects the probability to buy (if p is negative just put 0)

from pyspark.sql.functions import col, when, round

# Define the buy probability formula:
# P = (income - 3 * price) / income
# If P < 0, set it to 0

df = df.withColumn("buy_prob",round(
        when((col("annual_income") - 3 * col("price")) / col("annual_income") < 0, 0)
        .otherwise((col("annual_income") - 3 * col("price")) / col("annual_income")),
        2))

df.select("annual_income", "price", "buy_prob").show(10)

# Data Science (Using the PySpark Library)

## 4. Inferential Statistics

- **PySpark alone doesn’t offer all statistical tests natively, so we'll use some pandas as well.**

- [x] **Use a $\chi^2$ test to compare gender and body style.**

In [ ]:
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(pdf["gender"], pdf["body_style"])
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi² Test (Gender vs Body Style):")
print(f"Chi² = {chi2:.2f}, p-value = {p:.4f}")

- [x] **Use the t-test to compare the genders**

In [ ]:
from scipy.stats import ttest_ind

male_prices = pdf[pdf["gender"] == "Male"]["price"]
female_prices = pdf[pdf["gender"] == "Female"]["price"]

t_stat, p_value = ttest_ind(male_prices, female_prices, equal_var=False)

print("\nT-Test (Price: Male vs Female):")
print(f"t-statistic = {t_stat:.2f}, p-value = {p_value:.4f}")

- [x] **Use the ANOVA to compare prices across dealer regions.**

In [ ]:
from scipy.stats import f_oneway

region_groups = [group["price"].values for name, group in pdf.groupby("dealer_region")]
anova_stat, anova_p = f_oneway(*region_groups)

print("\nANOVA (Price by Dealer Region):")
print(f"F-statistic = {anova_stat:.2f}, p-value = {anova_p:.4f}")

## 5. Principal Component Analysis (PCA)

- [x] **Reduce the dimensionality of the dataset while keeping the most important information.**

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
import matplotlib.pyplot as plt
import pyspark.sql.functions as F

cols_to_drop = ['price', 'car_id', 'date', 'customer_name', 'phone']
df_clean = df.drop(*cols_to_drop)

categorical_cols = [col for col, dtype in df_clean.dtypes if dtype == 'string']
numerical_cols = [col for col, dtype in df_clean.dtypes if dtype != 'string']

indexers = [StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="skip") for col in categorical_cols]
encoders = [OneHotEncoder(inputCol=col + "_index", outputCol=col + "_ohe") for col in categorical_cols]

feature_cols = [col + "_ohe" for col in categorical_cols] + numerical_cols
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)

pca = PCA(k=2, inputCol="scaled_features", outputCol="pca_features")

pipeline = Pipeline(stages=indexers + encoders + [assembler, scaler, pca])
model = pipeline.fit(df_clean)
pca_result = model.transform(df_clean)

from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

get_pc1 = udf(lambda v: float(v[0]), DoubleType())
get_pc2 = udf(lambda v: float(v[1]), DoubleType())

pca_df = pca_result.withColumn("pc1", get_pc1("pca_features")) \
                   .withColumn("pc2", get_pc2("pca_features"))

pca_df = pca_df.withColumn("row_index", F.monotonically_increasing_id())
price_df = df.select("price").withColumn("row_index", F.monotonically_increasing_id())
pca_with_price = pca_df.join(price_df, on="row_index").select("pc1", "pc2", "price")

pandas_data = pca_with_price.toPandas()

plt.figure(figsize=(8, 6))
plt.scatter(pandas_data['pc1'], pandas_data['pc2'], c=pandas_data['price'], cmap='viridis', alpha=0.5)
plt.title("PCA: Car Sales Dataset (PySpark)")
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(label='Price ($)')
plt.show()


## 6. KMeans Clustering

- [x] **Use a KMeans clustering model to cluster cars based on price into "low price" and "high price".**

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import col, when
import matplotlib.pyplot as plt

vec_assembler = VectorAssembler(inputCols=["price"], outputCol="features")
df_vector = vec_assembler.transform(df)

kmeans = KMeans(k=2, seed=42, featuresCol="features", predictionCol="cluster")
model = kmeans.fit(df_vector)
df_clustered = model.transform(df_vector)

cluster_avg = df_clustered.groupBy("cluster").avg("price").orderBy("avg(price)")
cluster_avg.show()

low_price_cluster = cluster_avg.first()["cluster"]

df_labeled = df_clustered.withColumn(
    "price_segment",
    when(col("cluster") == low_price_cluster, "Low Price").otherwise("High Price")
)

pandas_data = df_labeled.select("price", "cluster").toPandas()

plt.figure(figsize=(10, 5))
plt.scatter(pandas_data["price"], [0] * len(pandas_data), c=pandas_data["cluster"], cmap="Set2", s=100)
plt.title("KMeans Clustering of Car Prices (PySpark)")
plt.xlabel("Price ($)")
plt.yticks([])
plt.show()


## 7. Linear Regression

- [x] **Use a linear regression model to predict the car price using the Engine, Body Style, Color, Dealer Region, Car Model, Company**

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
import matplotlib.pyplot as plt

selected_cols = ["engine", "body_style", "color", "dealer_region", "model", "company", "price"]
df_lr = df.select(*selected_cols)

categorical_cols = ["engine", "body_style", "color", "dealer_region", "model", "company"]
indexers = [StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid='skip') for col in categorical_cols]
encoders = [OneHotEncoder(inputCol=col + "_index", outputCol=col + "_ohe") for col in categorical_cols]

ohe_cols = [col + "_ohe" for col in categorical_cols]
assembler = VectorAssembler(inputCols=ohe_cols, outputCol="features")

lr = LinearRegression(featuresCol="features", labelCol="price", predictionCol="prediction")

pipeline = Pipeline(stages=indexers + encoders + [assembler, lr])
train_data, test_data = df_lr.randomSplit([0.8, 0.2], seed=42)
model = pipeline.fit(train_data)
predictions = model.transform(test_data)

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction")

mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
mse = evaluator.evaluate(predictions, {evaluator.metricName: "mse"})
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared: {r2}")

pandas_data = predictions.select("price", "prediction").toPandas()

plt.figure(figsize=(10, 5))
plt.scatter(pandas_data["price"], pandas_data["prediction"])
plt.plot([pandas_data["price"].min(), pandas_data["price"].max()],
         [pandas_data["price"].min(), pandas_data["price"].max()],
         color='red', linestyle='--')
plt.title("Actual vs Predicted Car Prices (PySpark)")
plt.xlabel("Actual Prices ($)")
plt.ylabel("Predicted Prices ($)")
plt.show()


## 8. Logistic Regression

- [] **Use a Logistic Regression model to predict the car model based on Transmission, and Body Style.**

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import seaborn as sns
import matplotlib.pyplot as plt

indexer = StringIndexer(inputCol="Transmission", outputCol="label")

indexer_model = StringIndexer(inputCol="Model", outputCol="Model_index")
indexer_body_style = StringIndexer(inputCol="Body Style", outputCol="Body_Style_index")
indexer_dealer_region = StringIndexer(inputCol="Dealer_Region", outputCol="Dealer_Region_index")

encoder_model = OneHotEncoder(inputCol="Model_index", outputCol="Model_vec")
encoder_body_style = OneHotEncoder(inputCol="Body_Style_index", outputCol="Body_Style_vec")
encoder_dealer_region = OneHotEncoder(inputCol="Dealer_Region_index", outputCol="Dealer_Region_vec")

assembler = VectorAssembler(inputCols=["Model_vec", "Body_Style_vec", "Dealer_Region_vec"], outputCol="features")

log_reg = LogisticRegression(featuresCol="features", labelCol="label")

pipeline = Pipeline(stages=[indexer, indexer_model, indexer_body_style, indexer_dealer_region, 
                            encoder_model, encoder_body_style, encoder_dealer_region, assembler, log_reg])

train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)
model = pipeline.fit(train_data)

predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy}")

y_true = predictions.select("label").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_true, y_pred)
cr = classification_report(y_true, y_pred)

print("Classification Report:\n", cr)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix - Logistic Regression")
plt.show()


## 9. Gradient Boost

- [x] **Use a gradient boosting model to predict a customer's gender based on their Annual Income, Car Model, Body Style, and Dealer Region.**

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

features = ["annual_income", "model", "body_style", "dealer_region"]
target = "gender"

indexers = [StringIndexer(inputCol=col, outputCol=col+"_idx", handleInvalid="keep") for col in features if col != "annual_income"]
encoders = [OneHotEncoder(inputCol=col+"_idx", outputCol=col+"_ohe") for col in features if col != "annual_income"]

ohe_cols = [col+"_ohe" for col in features if col != "annual_income"]
input_features = ohe_cols + ["annual_income"]
assembler = VectorAssembler(inputCols=input_features, outputCol="features")

pipeline = Pipeline(stages=indexers + encoders + [assembler])
df_prepped = pipeline.fit(df).transform(df)

pandas_df = df_prepped.select("features", target).toPandas()

X = np.array(pandas_df["features"].tolist())
y = pandas_df[target].values

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

report = classification_report(y_test, y_pred, target_names=le.classes_)
matrix = confusion_matrix(y_test, y_pred)

print("Classification Report:\n", report)

plt.figure(figsize=(8, 6))
sns.heatmap(matrix, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel("Predicted Gender")
plt.ylabel("Actual Gender")
plt.title("Confusion Matrix - XGBoost Gender Prediction")
plt.show()


## 10. Neural Networks

- [x] **Use a classification neural networks model to predict the "will buy" column (that you'll create)**

In [ ]:
from pyspark.sql.functions import when
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

df = df.withColumn("will_buy", when(df["price"] > 30000, 1).otherwise(0))

features = ["engine", "body_style", "color", "company", "annual_income"]
target = "will_buy"

indexers = [StringIndexer(inputCol=col, outputCol=col + "_idx", handleInvalid="keep") for col in features if col != "annual_income"]
encoders = [OneHotEncoder(inputCol=col + "_idx", outputCol=col + "_ohe") for col in features if col != "annual_income"]

ohe_cols = [col + "_ohe" for col in features if col != "annual_income"]
input_cols = ohe_cols + ["annual_income"]
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

pipeline = Pipeline(stages=indexers + encoders + [assembler])
df_prepped = pipeline.fit(df).transform(df)

pandas_df = df_prepped.select("features", "will_buy").toPandas()

X = np.array(pandas_df["features"].tolist())
y = np.array(pandas_df["will_buy"].tolist())

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title("Accuracy over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Loss over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.tight_layout()
plt.show()


# SQL Queries (Using the PySpark Library)

- **This csv file has over 23000 rows, therefore I decided to do the SQL queries here as opposed to uploading a file to MySQL.**

- [x] **Start by fixing the columns to SQL-friendly names (again).**

In [ ]:
# df = spark.read.csv("/kaggle/input/car-sales/car_sales.csv", header=True, inferSchema=True)

df = df.withColumnRenamed('Car_id', 'car_id') \
       .withColumnRenamed('Customer Name', 'customer_name') \
       .withColumnRenamed('Annual Income', 'annual_income') \
       .withColumnRenamed('Dealer_Name', 'dealer_name') \
       .withColumnRenamed('Company', 'company') \
       .withColumnRenamed('Model', 'model') \
       .withColumnRenamed('Engine', 'engine') \
       .withColumnRenamed('Transmission', 'transmission') \
       .withColumnRenamed('Color', 'color') \
       .withColumnRenamed('`Price ($)`', 'price') \
       .withColumnRenamed('Dealer_No', 'dealer_no') \
       .withColumnRenamed('Body Style', 'body_style') \
       .withColumnRenamed('Phone', 'phone') \
       .withColumnRenamed('Dealer_Region', 'dealer_region') \
       .withColumnRenamed('Date', 'date') \
       .withColumnRenamed('Gender', 'gender')



print(df.columns)

- [x] **Create a SQL query that retrieves the total number of cars sold.**

In [ ]:
df.createOrReplaceTempView("car_sales")

total_cars_sold = spark.sql("select count(*) as total_cars_sold from car_sales")
total_cars_sold.show()


- [x] **Create a SQL query that retrieves the total sales by dealer.**

In [ ]:
total_sales_by_dealer = spark.sql("""
    select dealer_name, sum(price) as total_sales
    from car_sales
    group by dealer_name
    order by total_sales desc
""")
total_sales_by_dealer.show()

- [x] **Top 10 companies that sold the most cars.**

In [ ]:
spark.sql("""
    select company, count(*) as cars_sold
    from car_sales
    group by company
    order by cars_sold desc
    limit 10
""").show()

- [x] **Top 10 most bought models.**

In [ ]:
spark.sql("""
    select model, count(*) as bought
    from car_sales
    group by model
    order by count desc
    limit 10
""").show()


- [x] **Pick the 2nd and 3rd most bought car colors.**

In [ ]:
# spark.sql("""
#     select color, count(*) as color_count
#     from car_sales
#     group by color
#     order by color_count desc
#     limit 1,2
# """).show()                     

# This limit function (skipping the 1st and showing the subsequent two) works in MySQL but not with PySpark

spark.sql("""
    select color, color_count
    from (select color, count(*) as color_count,
        row_number() over (order by count(*) desc) as rank
        from car_sales
        group by color) 
    where rank in (2,3)
""").show()                      # I used a subquery here instead of a cte (common table expression)

- [x] **Most sold cars by company and model.**

In [ ]:
spark.sql("""
    select company, model, count(*) as total_sold
    from car_sales
    group by company, model
    order by total_sold desc, company desc
    limit 10
""").show()

- [x] **Find the top 5 dealers with the highest total sales, along with their most sold car model and the number of times it was sold.**

In [ ]:
spark.sql("""
    with dealer_sales as (
        select dealer_name, model, count(*) as model_count,
        sum(price) over (partition by dealer_name) as total_sales,
        row_number() over (partition by dealer_name order by count(*) desc) as rank
        from car_sales
        group by dealer_name, model)
    select dealer_name, total_sales, model as most_sold_model, model_count
    from dealer_sales
    where rank in (1,2,3,4,5)
    order by total_sales desc
""").show()


- [x] **Find the average price of cars for each dealer region, but only for dealers who have sold at least 50 cars.**

In [ ]:
spark.sql("""
    with region_sales as (
        select dealer_region, count(*) as total_cars_sold, avg(price) as avg_price
        from car_sales
        group by dealer_region)
    select dealer_region, avg_price
    from region_sales
    where total_cars_sold >= 50
    order by avg_price desc
""").show()


- [x] **Find the top 3 customers who have spent the most money, and show the models they purchased along with total amount spent.**

In [ ]:
spark.sql("""
    with customer_spending as (
        select customer_name, sum(price) as total_spent,
        collect_list(model) as models_purchased
        from car_sales
        group by customer_name)
    select customer_name, total_spent, array_join(models_purchased, ', ') as models
    from customer_spending
    order by total_spent desc
    limit 3
""").show()
